In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# from fastai.vision import *
# from fastai.metrics import error_rate
from pathlib import Path
import numpy as np
# from fastai.vision.core import imagenet_stats
from fastai.vision.data import ImageDataLoaders
from fastai.vision.augment import aug_transforms

from fastai.vision import *
from fastai.vision.all import *
from fastai.metrics import error_rate


In [3]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [4]:
path=Path('Candle Data')
path_save=Path('Candle Data/Processed')

In [5]:
path.ls()

(#4) [Path('Candle Data/Up'),Path('Candle Data/Down'),Path('Candle Data/Proceed'),Path('Candle Data/models')]

In [6]:
np.random.seed(42)
data = ImageDataLoaders.from_folder(path, train=".ipynb_checkpoints/Training the Model and Inference-checkpoint.ipynb", valid_pct=0.2,
        ds_tfms=aug_transforms(flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.5,
                               max_rotate=3),
                                    batch_tfms=Normalize.from_stats(*imagenet_stats),
                                  size=224, num_workers=4)

In [7]:
print(data)

<h1> Training the Model </h1>

In [8]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn.data = data
print(learn.data)

In [9]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate,time
0,nan,0.000216,0.000000,00:03
1,nan,0.000216,0.000000,00:03
2,nan,0.000216,0.000000,00:02
3,nan,0.000216,0.000000,00:03


/home/siddhesh/.local/lib/python3.8/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [10]:
learn.unfreeze()

In [11]:
learn.lr_find()

ZeroDivisionError: integer division or modulo by zero

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-5,1e-4))

In [ ]:
learn.save('First Model')

In [ ]:
data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.,max_rotate=3), size=352, num_workers=4).normalize(imagenet_stats)

In [ ]:
learn.data=data

In [ ]:
gc.collect()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-5,1e-4))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-6,1e-5))

In [ ]:
learn.save('Model 2')

In [ ]:
correcti=0
list_down=[]
list_up=[]
for i in range(len(data.valid_ds)):
    prediction=learn.predict(data.valid_ds[i][0])
    if str(data.valid_ds[i][1])=="Down" and bool(prediction[1]==0) :
        correcti=correcti+1
        if prediction[2][0] >= 0.75 :
            list_down=list_down + [i]
            #print(prediction)
            #print()
    if str(data.valid_ds[i][1])=="Up" and bool(prediction[1]==1) :
        correcti=correcti+1
        if prediction[2][1] >= 0.75 :
            list_up=list_up + [i]
            #print(prediction)
            #print()
        

In [ ]:
len(list_down),len(list_up)

<h1> Finding the patterns </h1>

In [ ]:
idx=233
x,y = data.valid_ds[idx]
x.show()
print(data.valid_ds.y[idx])

In [ ]:
xb,_ = data.one_item(x)
xb_im = Image(data.denorm(xb)[0])
xb = xb.cuda()

In [ ]:
m = learn.model.eval()

In [ ]:
from fastai.callbacks.hooks import *

In [ ]:
def hooked_backward(cat=y):
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(xb)
            preds[0,int(cat)].backward()
    return hook_a,hook_g

In [ ]:
hook_a,hook_g = hooked_backward()

In [ ]:
acts  = hook_a.stored[0].cpu()
acts.shape

In [ ]:
avg_acts = acts.mean(0)
avg_acts.shape

In [ ]:
def show_heatmap(hm):
    fig,ax = plt.subplots()
    xb_im.show(ax)
    ax.imshow(hm, alpha=0.6, extent=(0,352,352,0),
              interpolation='bilinear', cmap='magma');
    return fig

In [ ]:
answer=show_heatmap(avg_acts)
answer.savefig(path/'trial.png')

In [ ]:
list_up

In [ ]:
for idx in list_up :    
    x,y = data.valid_ds[idx]
    x.show()
    print(data.valid_ds.y[idx])
    xb,_ = data.one_item(x)
    xb_im = Image(data.denorm(xb)[0])
    xb = xb.cuda()
    m = learn.model.eval()
    hook_a,hook_g = hooked_backward()
    acts  = hook_a.stored[0].cpu()
    print(acts.shape)
    avg_acts = acts.mean(0)
    print(avg_acts.shape)
    answer=show_heatmap(avg_acts)
    answer.savefig(path_save/'Up/{0}.png'.format(idx))
    print(idx)